In [4]:
from torchsummary import summary
import torch
from collections import deque
from unityagents import UnityEnvironment
import numpy as np
from dqn_agent import Agent
from model import QNetwork

# Question for banana navigation

i have Several questions when i read the code. I listed them below.

Q1:why we need two model in dqn_agent:qnetwork_target & qnetwork_local; how is soft_update function work?

Q2:why loss is calculated by the mean square difference of Q_expected, Q_targets?  
     from my understanding, Q_expected calculate the reward of current selected action and Q_targets calculate the reward of current and next state, which is similar to [TD controlis](https://classroom.udacity.com/nanodegrees/nd893/parts/8f607726-757e-4ef5-8b64-f2368755b89a/modules/a85374fa-6a60-425b-a480-85b211c5bd5d/lessons/78acdf66-f45c-418b-bbe1-30fda9e42c8c/concepts/b861cc26-d9fc-48f4-97bd-5b5aade565f9), is that means we want the current action reward approximately equal to the final reward of this episode?
     
Q3:can i regard the Q_value: **Q(St,At)** as the reward value? 


# Deep Q-learning Network for banana navigation

***In this project, the goal of the agent is to collect yellow banana and avoid blue banana.***

Given a state as a vector in length=37(containing the agent's velocity, along with ray-based perception of objects around agent's forward direction), the agent are expected to select the action that bring biggest **current** reward.  
Although the state is discrete and not very large(37), sampling the states will influence the performance. Thus, DQN is sellected as the method for this project.

**how dose DQN work?**  
- feed forward  
> taking a state value as input, the feed-forward QNetwork(in this project, a three layers' fully connected network) output the values on all actions. the action is selected with epsilon-greedy method. With probability $\epsilon$ the agent randomly selected an action, and with prob (1- $\epsilon$), the agent select the action with maxium output value. 

- back propagation
> loss is calculated by the mean square difference of Q_expected, Q_targets. Q_expected is the network's value of the selected action while Q_targets is the reward of current action plus the Q_targets_next with discount rate $\gamma$. I haven't totally understand why we doing this..., so i left some place here.

- hyprer parameters
>  epsilon-greedy: $\epsilon$ is decreaing with the increase of epoch, which means the agent tend to be more confinent of the action with the maxium value 




# Visualize the layer of the model and the curve while training

In [13]:
state_size = 37
action_size = 4
seed = 0
model = QNetwork(state_size, action_size, seed)
summary(model, input_size=(37,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]           4,864
            Linear-2                   [-1, 64]           8,256
            Linear-3                    [-1, 4]             260
Total params: 13,380
Trainable params: 13,380
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


## here we plot the episode-reward curve durning training

![Aaron Swartz](https://raw.githubusercontent.com/YCyuchen/RL_Navigation_Unity/master/score-episode-curve.png)